In [20]:
import random

suits = ['Hearts', 'Diamonds', 'Spades', 'Clubs'] 
#ranks = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
ranks= ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}
balance= 1000
chip_bal= 0
max_players= 1

In [21]:
class Card(object):
    def __init__(self,suit,rank): 
        self.suit=suit
        self.rank=rank
        
        #if rank=='Ace':
        if rank=='A':
            self.point = 11
        #elif rank in ['Jack', 'Queen','King']:
        elif rank in ['J', 'Q','K']:
            self.point = 10
        else:
            self.point = int(rank)
            #self.point= values.keys()
        self.hidden= False
        
    def __str__(self):
        if self.hidden:
            return '----'
        else:
            return self.rank + ' of ' + self.suit
    
    def hide_card(self):
        self.hidden = True

    def reveal_card(self):
        self.hidden = False

    def if_ace(self):
        return self.rank == 'A'
    

In [22]:
class Deck(object):
    def __init__(self):
        self.cards=[]
        for suit in suits:
            for rank in ranks:
                card=Card(suit,rank)
                self.cards.append(card)
                self.shuffle()
                
    def shuffle (self):
        random.shuffle (self.cards)
        
    def deal_card(self):
        if len(self.cards) == 0:
              return None
        else:
              return self.cards.pop(0)
            
    def __str__(self):
        cards_in_deck = ''
        for card in self.cards:
            cards_in_deck = cards_in_deck + str(card) + ' '
        return cards_in_deck


In [23]:
class Hand(object):
    def __init__(self):
        self.hand = [] 
        
    def add_card(self, card):
        self.hand.append(card)
        return self.hand

    def get_value(self):
        aces = 0
        value = 0
        for card in self.hand:
            if card.if_ace():
                aces += 1
            value += card.point
        while (value > 21) and aces:
            value -= 10
            aces -= 1
        return value

In [24]:
class Dealer(Hand):
    def __init__(self, name, deck):
        Hand.__init__(self)
        self.name = name
        self.deck = deck
        self.isBust = False

    def show_hand(self):
        for card in self.hand:
            print (card)
        print()

    def hit(self):
        print (" Hit ")
        self.add_card(self.deck.deal_card())
        return self.hand

    def stand(self):
        print ("%s gets %d. Done." % (self.name, self.get_value()))

    def check_bust(self):
        if self.get_value() > 21:
            self.isBust = True
            print ("%s BUST!" % self.name)
        else:
            self.stand()
    

In [25]:
class Player(Dealer):
    def __init__(self, name, deck, bet):
        Dealer.__init__(self, name, deck)
        self.bet = bet
        self.isBust = False
        self.isSurrender= False
        self.split = []
    


In [26]:
def play(player, deck):
    print (player.name + ':')
    player.show_hand()
    if player.name == 'Dealer':
        while player.get_value() < 17:
            player.hit()
            player.show_hand()
        player.check_bust()
    else:
        global chip_bal
        if chip_bal > player.bet  :
            if player.hand[0].point == player.hand[1].point:
                choice = input("Hit, Stand, Surrender? (h/s/u) ")
            else:
                choice = input("Hit, Stand, Surrender? (h/s/u) ")
        else:
            choice = input("Hit, Stand or Surrender? (h/s/u) ")
        
        while choice == 'h':
            player.hit()
            player.show_hand()
            if player.get_value() > 21:
                player.isBust = True
                print ("%s --> bust!" % player.name)
                break
            choice = input("Hit or Stand? (h/s) ")

        if choice == 's':
            player.stand()

        if choice == 'u':
            player.isSurrender = True
            chip_bal += (player.bet - player.bet / 2)
            print ("New balance = %d" % chip_bal)


In [27]:
def report(player, dealer):
    global chip_bal
    if player.isSurrender:
        tag = 'surrender'
    elif player.isBust:
        tag = 'lose'
    elif len(player.hand) == 2 and player.get_value() == 21:
        tag = 'blackjack'
        chip_bal += player.bet * 3
    elif dealer.isBust or (player.get_value() > dealer.get_value()):
        tag = 'win'
        chip_bal += player.bet * 2
    elif player.get_value() == dealer.get_value():
        tag = 'push'
        chip_bal += player.bet
    else:
        tag = 'lose'
    print ("%s: %-*s Balance = %d" % (player.name, 10, tag, chip_bal))


In [28]:
def game():
    players = []
    global chip_bal
    deck = Deck()

    #player_num = int(input('Enter the num of players '))
    player_num=1
    
    print ("\nLet's get started...\n")
    for i in range(player_num):
        if chip_bal > 0:
            player_name = 'Player'+str(i+1)
            print ("%s:" % player_name)
            player_bet = int(input("Please bet. The minimal bet is 1 chip. "))
            chip_bal -= player_bet
            print( "Balance updated. New balance is %d." % chip_bal)
            player = Player(player_name, deck, player_bet)
            players.append(player)
        else:
            print ("\nThe actual number of player is %d. There's no balance to support more players." % (len(players)))
            break

    dealer = Dealer('Dealer', deck)

    for i in range(2):
        for player in (players + [dealer]):
            player.add_card(deck.deal_card())

    dealer.hand[1].hide_card()
    print ("\nDealer:")
    dealer.show_hand()
    print()
    dealer.hand[1].reveal_card()

    for player in (players + [dealer]):
        play(player, deck)
        print()

    print ("...RESULT...\n")

    for player in players:
        if not player.split:
            report(player, dealer)
        else:
            print ("%s: split" % player.name)
            for p in player.split:
                report(p, dealer)

    print ("\nFinal chip balance is %d.\n" % chip_bal)


In [29]:
if __name__ == '__main__':

    chip_bal = int(input("\nWelcome to BlackJack! Please enter the chip balance: (1-1000) "))
    while True:
        game()
        if chip_bal < 1:
            print ("You don't have enough balance to proceed. Game over.")
            break
        proceed = input("Do you want to continue? (y/n) ")
        if proceed == 'n':
            print ("\nThank you for playing! See you next time.")
            break


Welcome to BlackJack! Please enter the chip balance: (1-1000) 500

Let's get started...

Player1:
Please bet. The minimal bet is 1 chip. 50
Balance updated. New balance is 450.

Dealer:
7 of Spades
----


Player1:
2 of Hearts
7 of Diamonds

Hit, Stand, Surrender? (h/s/u) h
 Hit 
2 of Hearts
7 of Diamonds
J of Diamonds

Hit or Stand? (h/s) s
Player1 gets 19. Done.

Dealer:
7 of Spades
Q of Diamonds

Dealer gets 17. Done.

...RESULT...

Player1: win        Balance = 550

Final chip balance is 550.

Do you want to continue? (y/n) n

Thank you for playing! See you next time.
